In [1]:
import json
import threading
import time
import uuid
import paho.mqtt.client as paho
from paho import mqtt
from datetime import datetime

In [2]:
# Função para ler o arquivo de configuração
def read_config(filepath):
    with open(filepath, 'r') as file:
        config = json.load(file)
    return config

# Função para obter o uso da CPU
def get_cpu_usage():
    try:
        with open('/proc/stat', 'r') as f:
            lines = f.readlines()
    except IOError:
        print("Error opening /proc/stat")
        return -1.0

    for line in lines:
        if line.startswith('cpu '):
            cpu_times = line.split()[1:]
            cpu_times = list(map(int, cpu_times))
            if len(cpu_times) < 4:
                return -1.0
            idle_time = cpu_times[3]
            total_time = sum(cpu_times)
            cpu_usage = 1.0 - (idle_time / total_time)
            cpu_usage *= 100.0
            return cpu_usage
    return -1.0

# Função para obter o uso da RAM
def get_ram_usage():
    try:
        with open('/proc/meminfo', 'r') as meminfo:
            m_total = 0
            m_avail = 0
            for line in meminfo:
                if line.startswith("MemTotal:"):
                    m_total = int(line.split()[1])
                elif line.startswith("MemAvailable:"):
                    m_avail = int(line.split()[1])
            if m_total == 0:
                raise ValueError("MemTotal is zero, unable to compute RAM usage.")
            return ((m_total - m_avail) / m_total) * 100.0
    except FileNotFoundError:
        print("Error opening /proc/meminfo")
        return -1.0
    except Exception as e:
        print(f"Error: {e}")
        return -1.0

# Função para obter o timestamp atual
def get_timestamp():
    now = datetime.utcnow()
    timestamp = now.strftime("%Y-%m-%dT%H:%M:%SZ")
    return timestamp

# Funções de callback do MQTT
def on_connect(client, userdata, flags, rc, properties=None):
    print("CONNACK received with code %s." % rc)

def on_publish(client, userdata, mid, properties=None):
    # print("mid: " + str(mid))
    pass

# Publica uma mensagem inicial
def publish_initial_message(client, machine_id, sensors):
    initial_message = {
        "machine_id": machine_id,
        "sensors": sensors
    }
    client.publish("/sensor_monitors", json.dumps(initial_message), qos=1)

# Função para publicar dados do sensor
def publish_sensor_data(client, machine_id, sensor_id, get_data_func, interval):
    while True:
        data = {
            "timestamp": get_timestamp(),
            "value": get_data_func()
        }
        topic = f"/sensors/{machine_id}/{sensor_id}"
        client.publish(topic, json.dumps(data), qos=1)
        time.sleep(interval / 1000)

In [3]:
config = read_config('config.json')


In [4]:
# Configuração do cliente MQTT
client = paho.Client(client_id="", userdata=None, protocol=paho.MQTTv5)
client.on_connect = on_connect

/tmp/ipykernel_14326/3489700255.py:2: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = paho.Client(client_id="", userdata=None, protocol=paho.MQTTv5)


In [5]:
client.tls_set(tls_version=mqtt.client.ssl.PROTOCOL_TLS)
client.username_pw_set(config['mqtt']['username'], config['mqtt']['password'])
client.connect(config['mqtt']['broker_url'], config['mqtt']['broker_port'])

<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

In [6]:
client.on_publish = on_publish


In [7]:
# Iniciar threads para cada sensor
threads = []
for sensor in config['sensors']:
    sensor_id = sensor['sensor_id']
    data_interval = sensor['data_interval']
    if sensor_id == "cpu_usage":
        get_data_func = get_cpu_usage
    elif sensor_id == "ram_usage":
        get_data_func = get_ram_usage
    else:
        continue

    thread = threading.Thread(target=publish_sensor_data, args=(client, config['machine_id'], sensor_id, get_data_func, data_interval))
    threads.append(thread)
    thread.start()

{'timestamp': '2024-07-29T23:02:16Z', 'value': 14.977505482332198}
{'timestamp': '2024-07-29T23:02:16Z', 'value': 35.80377001573919}


In [8]:
publish_initial_message(client, config['machine_id'], config['sensors'])
client.loop_forever()

CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code Success.
CONNACK received with code S

{'timestamp': '2024-07-29T23:02:21Z', 'value': 36.08259255701453}{'timestamp': '2024-07-29T23:02:21Z', 'value': 14.953222888269346}

{'timestamp': '2024-07-29T23:02:26Z', 'value': 36.34038025286151}
{'timestamp': '2024-07-29T23:02:26Z', 'value': 14.930732721688988}
{'timestamp': '2024-07-29T23:02:31Z', 'value': 35.46332079617136}
{'timestamp': '2024-07-29T23:02:31Z', 'value': 14.915566531142343}
{'timestamp': '2024-07-29T23:02:36Z', 'value': 35.272874162855025}
{'timestamp': '2024-07-29T23:02:36Z', 'value': 14.893338860367978}
{'timestamp': '2024-07-29T23:02:41Z', 'value': 14.865401792099886}
{'timestamp': '2024-07-29T23:02:41Z', 'value': 35.39120132613124}
{'timestamp': '2024-07-29T23:02:46Z', 'value': 14.842757331389912}
{'timestamp': '2024-07-29T23:02:46Z', 'value': 35.264992253561026}
{'timestamp': '2024-07-29T23:02:51Z', 'value': 35.2717904003271}
{'timestamp': '2024-07-29T23:02:51Z', 'value': 14.816721679116851}
{'timestamp': '2024-07-29T23:02:56Z', 'value': 35.97229508883158}{'t

KeyboardInterrupt: 